# 🚀 Comprehensive Model Improvements for Chest X-ray Classification

## 📋 Tổng Quan (Overview)

Notebook này triển khai **toàn bộ improvement plan** được thiết kế để cải thiện hiệu suất của các mô hình phân loại bệnh từ ảnh X-quang ngực. Chúng ta sẽ áp dụng các kỹ thuật state-of-the-art dựa trên phân tích sâu các hạn chế của mô hình gốc.

### 🎯 Mục Tiêu Cải Thiện

| Model | Original AUC | Target AUC | Expected Gain |
|-------|--------------|------------|---------------|
| ResNet-34 | 0.86 | 0.88-0.89 | +2-3% |
| ViT-Base | 0.86 | 0.88-0.90 | +2-4% |
| Swin Transformer | - | 0.89-0.91 | New |
| Ensemble | - | 0.90-0.92 | Best |

### 📊 Các Vấn Đề Cần Giải Quyết

1. **Training from Scratch**: Mô hình gốc không sử dụng pre-trained weights → Mất đi kiến thức học được từ ImageNet
2. **Class Imbalance**: Dataset cực kỳ mất cân bằng (No Finding: 53.84% vs Hernia: 0.20%) → Bias về các class phổ biến
3. **Weak Augmentation**: Chỉ dùng flip & rotate → Generalization kém
4. **Fixed Architecture**: Không tối ưu hóa architecture → Bỏ lỡ modern techniques
5. **Label Noise**: NIH dataset có ~10% label errors từ NLP extraction → Học sai patterns

## 🔧 Setup & Dependencies

### Giải Thích
Chúng ta cần các thư viện sau:
- **timm**: SOTA vision models với pre-trained weights
- **albumentations**: Advanced augmentation cho medical imaging
- **sklearn**: Metrics và utilities
- **torch**: Deep learning framework

In [1]:
# Standard libraries
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm
from typing import Dict, List, Tuple, Optional

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision
from torchvision import transforms

# Computer Vision
from PIL import Image
import cv2

# Advanced libraries
try:
    import timm  # PyTorch Image Models
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    print("✅ All advanced libraries loaded successfully")
except ImportError as e:
    print(f"❌ Missing libraries: {e}")
    print("🔧 Please install manually using:")
    print("   pip install timm albumentations")
    print("   Or activate your virtual environment and install there")
    print("\n⚠️  Skipping advanced features for now...")
    
    # Create dummy classes to avoid import errors
    class DummyAlbumentations:
        class Compose:
            def __init__(self, transforms):
                self.transforms = transforms
            def __call__(self, **kwargs):
                return kwargs
        
        class Resize:
            def __init__(self, *args, **kwargs):
                pass
        class RandomCrop:
            def __init__(self, *args, **kwargs):
                pass
        class HorizontalFlip:
            def __init__(self, *args, **kwargs):
                pass
        class ShiftScaleRotate:
            def __init__(self, *args, **kwargs):
                pass
        class OneOf:
            def __init__(self, transforms, *args, **kwargs):
                self.transforms = transforms
        class GaussNoise:
            def __init__(self, *args, **kwargs):
                pass
        class GaussianBlur:
            def __init__(self, *args, **kwargs):
                pass
        class MotionBlur:
            def __init__(self, *args, **kwargs):
                pass
        class RandomBrightnessContrast:
            def __init__(self, *args, **kwargs):
                pass
        class CLAHE:
            def __init__(self, *args, **kwargs):
                pass
        class GridDistortion:
            def __init__(self, *args, **kwargs):
                pass
        class Normalize:
            def __init__(self, *args, **kwargs):
                pass
    
    A = DummyAlbumentations()
    
    class DummyToTensorV2:
        def __init__(self):
            pass
        def __call__(self, image):
            return image
    
    ToTensorV2 = DummyToTensorV2
    
    # Dummy timm
    class DummyTimm:
        def create_model(self, *args, **kwargs):
            raise NotImplementedError("timm not available - install with: pip install timm")
    
    timm = DummyTimm()
    
    print("✅ Using dummy implementations - limited functionality")

# Scikit-learn
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import StratifiedKFold

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
def set_seed(seed=42):
    """Đặt random seed cho reproducibility"""
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")

✅ All advanced libraries loaded successfully
🖥️  Device: cuda
   GPU: NVIDIA GeForce RTX 3060 Laptop GPU
   CUDA Version: 12.6


## 📁 Configuration & Paths

### Giải Thích
Centralized configuration giúp dễ dàng điều chỉnh hyperparameters và paths

In [2]:
# Project paths
PROJECT_ROOT = Path("D:/MSE/10.Deep Learning/Group_Final/ViT-Chest-Xray")
DATA_DIR = PROJECT_ROOT / "Project" / "data"
CSV_PATH = PROJECT_ROOT / "Project" / "input" / "Data_Entry_2017_v2020.csv"
IMAGE_DIR = PROJECT_ROOT / "Project" / "input" / "images"
SAVE_DIR = PROJECT_ROOT / "Project" / "improve" / "results"
SAVE_DIR.mkdir(exist_ok=True, parents=True)

# Training configuration
CONFIG = {
    # Data
    'img_size': 224,
    'num_classes': 15,
    'batch_size': 32,
    'num_workers': 4,
    
    # Training
    'epochs': 30,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'warmup_epochs': 3,
    
    # Augmentation
    'use_advanced_aug': True,
    'aug_probability': 0.5,
    
    # Class imbalance
    'use_weighted_loss': True,
    'use_focal_loss': True,
    'focal_alpha': 0.25,
    'focal_gamma': 2.0,
    
    # Transfer learning
    'use_pretrained': True,
    'freeze_backbone_epochs': 5,  # Freeze backbone for first N epochs
    
    # Advanced techniques
    'use_label_smoothing': True,
    'label_smoothing': 0.1,
    'use_mixup': True,
    'mixup_alpha': 0.2,
    
    # Model saving
    'save_best_only': True,
    'early_stopping_patience': 10,
}

# Disease classes
DISEASE_CLASSES = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass',
    'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema',
    'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'No Finding'
]

print("✅ Configuration loaded successfully")
print(f"📊 Number of classes: {CONFIG['num_classes']}")
print(f"🖼️  Image size: {CONFIG['img_size']}x{CONFIG['img_size']}")
print(f"📦 Batch size: {CONFIG['batch_size']}")

✅ Configuration loaded successfully
📊 Number of classes: 15
🖼️  Image size: 224x224
📦 Batch size: 32
